# Практическое задание №5 по теме "Задача оттока: варианты постановки, возможные способы решения".

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 
    * бустинг
    * логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2:
    * 1 доллар на привлечение (удержание)
    * 2 доллара - с каждого правильно классифицированного (True Positive) удержанного. 
    
подсказка: нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 

4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix

In [2]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Не самое плохое распределение (1 к 4)

In [4]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [5]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

# перевод категориального признака в бинарное представление
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [7]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler()) # делаем для логистической регрессии
            ])
    final_transformers.append((cont_col, cont_transformer))

In [8]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [9]:
# словарь для записи результатов 
models_results = {
    'model': [],
    'threshold': [],
    'f-score': [],
    'precision': [],
    'recall': []
}

## Задание №1

#### 1. Случайный лес

In [10]:
pipeline_rf = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

pipeline_rf.fit(X_train, y_train)
preds = pipeline_rf.predict_proba(X_test)[:, 1]

In [11]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
cnf_matrix_rf = confusion_matrix(y_test, preds > thresholds[ix])

models_results['model'].append('RandomForest')
models_results['threshold'].append(thresholds[ix])
models_results['f-score'].append(fscore[ix])
models_results['precision'].append(precision[ix])
models_results['recall'].append(recall[ix])

#### 2. Градиентный бустинг

In [12]:
pipeline_xgb = Pipeline([
    ('features', feats),
    ('classifier', XGBClassifier(random_state=42)),
])

pipeline_xgb.fit(X_train, y_train)
preds = pipeline_xgb.predict_proba(X_test)[:, 1]

In [13]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
cnf_matrix_xgb = confusion_matrix(y_test, preds > thresholds[ix])

models_results['model'].append('XGB')
models_results['threshold'].append(thresholds[ix])
models_results['f-score'].append(fscore[ix])
models_results['precision'].append(precision[ix])
models_results['recall'].append(recall[ix])

#### 3. Логистическая регрессия

In [14]:
pipeline_lr = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=42)),
])

pipeline_lr.fit(X_train, y_train)
preds = pipeline_lr.predict_proba(X_test)[:, 1]

In [15]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
cnf_matrix_lr = confusion_matrix(y_test, preds > thresholds[ix])

models_results['model'].append('LogReg')
models_results['threshold'].append(thresholds[ix])
models_results['f-score'].append(fscore[ix])
models_results['precision'].append(precision[ix])
models_results['recall'].append(recall[ix])

## Задание №2

In [16]:
pd.DataFrame(models_results).set_index('model')

,threshold,f-score,precision,recall
model,,,,
RandomForest,0.380000,0.641283,0.654397,0.628684
XGB,0.373318,0.625850,0.619231,0.632613
LogReg,0.289522,0.509700,0.462400,0.567780


По полученным результатам мы можем сделать вывод, что самые высокие результаты показала модель **RandomForest**.

## Задание №3

Сделаем оценку экономической эффективности для модели **RandomForest**.

In [17]:
TN = cnf_matrix_rf[0][0]
FP = cnf_matrix_rf[0][1]
FN = cnf_matrix_rf[1][0]
TP = cnf_matrix_rf[1][1]

In [18]:
income = TP * 2
costs = (FP + TP) * 1
profit = income - costs
print(f"Экономическая выгода: {profit}")

Экономическая выгода: 156


## Задание №4

In [19]:
params={'classifier__max_features':[0.3, 0.5, 0.7], # максимальное количество признаков (доля признаков)
        'classifier__min_samples_leaf':[1, 2, 3], # минимальное количество объектов в одном листе
        'classifier__max_depth':[50, 100, 150, 200], # максимальная глубина дерева в лесу
        'classifier__class_weight':[{0:1, 1:4}, 'balanced_subsample'] 
        }

In [20]:
grid = GridSearchCV(pipeline_rf,
                    param_grid=params,
                    cv=5,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__class_weight': {0: 1, 1: 4},
 'classifier__max_depth': 50,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 1}

Создаем новый pipeline и обучаем модель уже с новыми найденными GridSearch параметрами:

Создаем новый pipeline и обучаем модель уже с основыми параметрами:

In [21]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth=50, 
                                          max_features=0.3, 
                                          min_samples_leaf=1, # хоть это и переобучние, но у нас все таки лес - Bagging
                                          class_weight={0:1, 1:4},
                                          random_state=42))
])

pipeline.fit(X_train, y_train)
preds = pipeline.predict_proba(X_test)[:, 1]

In [22]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

models_results['model'].append('RandomForestOpt')
models_results['threshold'].append(thresholds[ix])
models_results['f-score'].append(fscore[ix])
models_results['precision'].append(precision[ix])
models_results['recall'].append(recall[ix])

Сравним получившиеся метрики с ранее посчитанными:

In [23]:
pd.DataFrame(models_results).set_index('model')

,threshold,f-score,precision,recall
model,,,,
RandomForest,0.380000,0.641283,0.654397,0.628684
XGB,0.373318,0.625850,0.619231,0.632613
LogReg,0.289522,0.509700,0.462400,0.567780
RandomForestOpt,0.420000,0.643923,0.703963,0.593320


## Задание №5

In [24]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]

In [25]:
income = TP * 2
costs = (FP + TP) * 1
profit_after_opt = income - costs
print(f"Экономическая выгода до подбора гиперпараметров: {profit}")
print(f"Экономическая выгода после подбора гиперпараметров: {profit_after_opt}")
print(f"Выгода от применения оптимизации: {profit_after_opt-profit}")

Экономическая выгода до подбора гиперпараметров: 156
Экономическая выгода после подбора гиперпараметров: 178
Выгода от применения оптимизации: 22


---